In [1]:
from src.utils import read_jsonl
from src.models.models.llava import MatplotlibEncoder
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from src.utils import read_jsonl, write_jsonl
from src.visualization.visualize import paper_mpl_env, title
import textwrap
import numpy as np
from tqdm import tqdm

/gscratch/bdata/mikeam/anaconda3/envs/TSandLang/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
pd.DataFrame(questions)["category"].value_counts()

category
time-series    87261
description    84862
Name: count, dtype: int64

In [3]:
questions = read_jsonl("/gscratch/bdata/datasets/llms_and_timeseries/All_MCQ.json")
questions[0]

{'description': ' A scientist is measuring the temperature in an industrial freezer over two weeks, with a new reading taken every 6 hours. An unexpected power outage occurs in the middle of this period lasting 3 days. This causes the freezer temperature to rise before it is restored and begins dropping again. ',
 'description_short': ' A temperature reading from an industrial freezer over two weeks, including a 3-day power outage. ',
 'description_tiny': ' Freezer Temperature Time Series ',
 'characteristics': ' \n1) Continuous increment of temperature during power outage, \n2) Slow decrease of temperature once power is restored until it reaches normal levels, \n3) Stable temperature values before the outage and after temperature restoration, \n4) Overall time series has slow ascending and descending trends during the outage and recovery period respectively,\n5) Lack of daily temperature variation due to constant artificial control.\n',
 'generator': ' \n```python\nimport numpy as np\

In [4]:
metadata_lookup = {q["uuid"]: q for q in questions}

In [10]:
questions = read_jsonl("/gscratch/bdata/datasets/llms_and_timeseries/All_MCQ.json")
N_TO_SAMPLE = 1000
np.random.shuffle(questions)

to_metadata = []

all_ids = set()
non_dup_questions = []
for q in questions:
    if q["uuid"] in all_ids:
        continue
    else:
        all_ids.add(q["uuid"])
        non_dup_questions.append(q)
i=0
i_start=0
n_plotted = 0
with paper_mpl_env():    
    for i in range(i_start, len(non_dup_questions)):
    # while n_plotted < N_TO_SAMPLE:
        scenario = non_dup_questions[i]
        fig, ax = plt.subplots(figsize=(6,4), dpi=100)
        try:
            uuid = scenario['uuid']
            ts = scenario['series']
            desciption = scenario['description'].strip()
            
            options = [x.strip() for x  in scenario['options']]
            true_answer = options[scenario['answer_index']]
            np.random.shuffle(options)
            answer_index = options.index(true_answer)

            desciption_tiny = title(scenario['description_tiny'])
            start = scenario['metadata']['start']
            end = scenario['metadata']['end']
            units = title(scenario['metadata']['units'])
        except KeyError:
            i+=1
            continue

        to_metadata.append({
            'uuid': uuid,
            'question': scenario["question"],
            "option_0": options[0],
            "option_1": options[1],
            "option_2": options[2],
            "option_3": options[3],
            "ts_quid": scenario["ts_qid"],
            "label": answer_index,
        })
        
        try:
            x = pd.date_range(start=start, end=end, periods=len(ts))
        except:
            i+=1
            continue
        ax.plot(x,ts)
        ax.set_title(scenario['description_tiny'])
        # Set xticks to just first and last
        ax.set_xticks([x[0], x[-1]])
        ax.set_ylabel(scenario['metadata']['units'])

        plt.tight_layout()
        plt.savefig(f"data/processed/to_retool/figs_600_400/{uuid}.png")
        i+=1
        n_plotted+=1

to_metadata_df = pd.DataFrame(to_metadata)
to_metadata_df.to_csv("data/processed/to_retool/metadata.csv", index=False)

/tmp/ipykernel_12440/2507423422.py:63: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_12440/2507423422.py:63: UserWarning: Glyph 179 (\N{SUPERSCRIPT THREE}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_12440/2507423422.py:64: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  plt.savefig(f"data/processed/to_retool/figs_600_400/{uuid}.png")
/tmp/ipykernel_12440/2507423422.py:64: UserWarning: Glyph 179 (\N{SUPERSCRIPT THREE}) missing from current font.
  plt.savefig(f"data/processed/to_retool/figs_600_400/{uuid}.png")
/tmp/ipykernel_12440/2507423422.py:63: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_12440/2507423422.py:64: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from current font.
  plt.savefig(f"data/processed/to_retool/figs_600_400/{uuid}.png")
/tmp/ipykernel_12440/2507423422.py:63: UserWarning: Tight layout not appli

In [6]:
to_metadata[0]["uuid"]

'38ce609a-b350-4dbe-aeb0-d57b20835253'

,uuid,question,option_0,option_1,option_2,option_3,ts_quid,label
0,38ce609a-b350-4dbe-aeb0-d57b20835253,What are the factors contributing to the trend...,The overall trend is contributed by decreasing...,The overall trend is contributed by stable pop...,The overall trend is contributed by increasing...,The overall trend is contributed by increasing...,16,3
1,80c0b3cf-ffbb-4bde-a5cd-8270c925e0dd,Are there any abrupt spikes in sales during th...,Abrupt spikes in sales during the festival are...,Yes as some days may hold special events assoc...,No abrupt spikes in sales during the festival ...,Abrupt spikes in sales during the festival are...,12,1
2,0b44e954-e9f5-4779-b44f-5c3105b35561,What is the approximate average sales value du...,Assuming an arbitrary average of 800 during co...,Assuming an arbitrary average of 900 during co...,Assuming an arbitrary average of 1100 during c...,Assuming an arbitrary average of 1000 during c...,5,3
3,b86f0817-752c-49c0-8b12-e6c58bef5327,How much rainfall was recorded on the first da...,5,4.5,6,7,4,2
4,d3bb84c9-b2d9-47ed-975f-3c5284231273,How was the water usage affected during the ev...,The water usage remained unchanged during the ...,The water usage showed a sudden decrease durin...,The water usage gradually decreased during the...,The water usage showed a sudden increase follo...,17,3
...,...,...,...,...,...,...,...,...
1002,8ae5c0f6-e4a7-4fb7-a04d-9b69812d240e,Approximately how much above the daily average...,Approximately 300 units above the daily average,Approximately 299 units above the daily average,Approximately 300,Approximately 301 units above the daily average,3,0
1003,b9968d10-93c9-48bf-85c1-5b0be100cb22,One week before finals what is the stress level?,Approximately 90.,Approximately 75.,Approximately 85.,Approximately 80.,9,3
1004,af582dc3-bf4d-4320-b9cf-a4013d948d84,What could lead to daily fluctuations in bambo...,Consumption may have periodic fluctuations day...,Consumption may have controlled fluctuations d...,Consumption may have fixed fluctuations daytod...,Consumption may have random fluctuations dayto...,14,3
1005,02ca273d-aaeb-4614-a521-5dc178052e0a,What was the highest number of umbrellas sold ...,3600,3400,3200,3800,0,0


## Version Two
Make metadata for the second version of questions

In [5]:
categorical_qa = read_jsonl("data/processed/mikes_qa.jsonl")
np.random.shuffle(categorical_qa)
N_TO_SAMPLE = len(categorical_qa) * 5

to_metadata = []
for scenario in categorical_qa[:N_TO_SAMPLE]:
     uuid = scenario['uuid']
     options = [x.strip() for x  in scenario['options']]
     true_answer = options[scenario['label_index']]
     np.random.shuffle(options)
     answer_index = options.index(true_answer)
     to_metadata.append({
          'uuid': uuid,
          'question': scenario["question"],
          'options': options,
          "option_0": options[0],
          "option_1": options[1],
          "option_2": options[2],
          "option_3": options[3],
          "category": scenario["category"],
          "label": answer_index,
          "series": metadata_lookup[uuid]["series"],
          "description_tiny": metadata_lookup[uuid]["description_tiny"],
          "metadata": metadata_lookup[uuid]["metadata"],
          

     })
     
to_metadata_df = pd.DataFrame(to_metadata)
to_metadata_df.to_csv("data/processed/to_retool/metadata_categorical.csv", index=False)
to_metadata_df.to_json("data/processed/mike_qa_processed.jsonl", orient="records", lines=True)

In [11]:
to_metadata_df

,uuid,question,options,option_0,option_1,option_2,option_3,category,label,series,description_tiny,metadata
0,07cbce1b-ddef-4a59-9aa0-3a0d9dc1cd5e,What might be the impact on water consumption ...,[The total water consumption would likely not ...,The total water consumption would likely not b...,The total water consumption would likely decre...,The total water consumption would likely remai...,The total water consumption would likely incre...,counterfactual_reasoning,1,"[0.0, 687.0, 1324.0, 2081.0, 2788.0, 3395.0, 4...",\nCity water consumption\n,"{'start': '2022-01-01', 'end': '2022-12-31', '..."
1,7ed2f3e6-7434-444e-93e9-e9f22d745239,What are some other scenarios where seasonal p...,[Seasonal patterns in office buildings where w...,Seasonal patterns in office buildings where we...,Seasonal patterns in agriculture where harvest...,Seasonal patterns in education where summer br...,Seasonal patterns in retail where holiday shop...,analogical_reasoning,3,"[6339.417154475185, 7987.555510995635, 6064.43...",EV charging station consumption,"{'start': '2023-01-01', 'end': '2023-12-31', '..."
2,8d55bc9f-5fb5-4480-8321-f8241ecf4212,Why might the bike-sharing service not require...,[More bikes should be added to compensate for ...,More bikes should be added to compensate for t...,More bikes should be added as winter is the pe...,An increase in bikes might not be necessary as...,An increase in bikes is essential as the data ...,argumentation,2,"[52268.90956317842, 40509.20001049973, 49500.8...",\nBike-sharing distances\n,"{'start': '2023-01-01', 'end': '2023-12-31', '..."
3,50c85c5e-fea3-4ce2-b524-28d1a36991cf,What are some reasons for the theme park's inc...,[The increase in popularity is likely due to a...,The increase in popularity is likely due to a ...,"The popularity likely decreased over time, con...",The increase in popularity is probably random ...,The general upward trend could be due to effec...,explanation,3,"[10000.0, 8561.846996699882, 11969.85582436364...",\nTheme Park Attendance\n,"{'start': '2021-01-01', 'end': '2021-12-31', '..."
4,7708ec2b-3763-45fe-8f3f-7446c967a764,What are some other scenarios where similar pa...,[Similar patterns are seen in the time series ...,Similar patterns are seen in the time series o...,Similar patterns are often seen in the stock m...,Similar patterns might be observed in the time...,Similar patterns are observed in the time seri...,analogical_reasoning,2,"[115.83242152594529, 119.4373317277367, 98.638...",Blood pressure monitoring,"{'start': '2022-01-10 00:00:00', 'end': '2022-..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3670,70f1b98b-6979-4978-9b87-86eb33f5116d,Can you think of similar scenarios where commu...,[Similar scenarios include increased purchases...,Similar scenarios include increased purchases ...,A similar scenario is the increase in electric...,A similar scenario is the decrease in umbrella...,A similar scenario is the consistent sale of s...,analogical_reasoning,0,"[10.727395785426726, 18.85624023282667, 8.0027...",\nMask Vending Sales\n,"{'start': '2022-01-01T00:00:00Z', 'end': '2022..."
3671,46fd0c1b-3c7c-4275-8bb7-b1148aa31789,What could be the reason for the observed drop...,[The observed drop could be a result of poor m...,The observed drop could be a result of poor mo...,The drop in sales could be due to the cinema's...,The drop in sales might be due to technical is...,The drop in sales might be attributed to weekd...,explanation,1,"[42.49277140375457, 0.0, 2.9761458759450727, 5...",\nCinema ticket sales\n,"{'start': '01/01/2023 00:00', 'end': '01/05/20..."
3672,fa346f9f-0603-4ace-8774-bcc288ced530,Can you think of similar scenarios where seaso...,"[During holiday seasons, e-commerce sites ofte...","During holiday seasons, e-commerce sites often...",Seasonal events in the fitness industry lead t...,"In the travel industry, seasonal events lead t...",Seasonal events have no impact on user behavio...,analogical_reasoning,0,"[6.014140798216324, 6.9533442178995